In [1]:
!pip install tiingo
!pip install tiingo[pandas]

In [2]:
# Importing necessary libraries 
import os
import json
import pandas as pd
import numpy as np
import glob
from IPython import get_ipython


# For data cleaning and visualization
import pandas as pd
import numpy as np
from numpy import array
from datetime import date
from datetime import datetime, timedelta
import pandas_datareader.data as web


# For saving the model
import pickle

# For model Evaluation
from sklearn.metrics import mean_squared_error, r2_score
from sklearn import metrics
import math

import re
import nltk
from nltk.corpus import stopwords

import nltk
# NLTK VADER for sentiment analysis
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [117]:
company = 'NIO'
dfname = "FinalData/news_headlines_"+str(company1)+".csv"

In [118]:
df = pd.read_csv(dfname, index_col=0)
df

,headline,date
0,Get SMS ing U S tech execs urge Web starved ...,2009-04-22
1,CORRECTED Little known cloud startups attrac...,2009-05-06
2,US STOCKS Wall St climbs on tech shares earni...,2009-07-16
3,US STOCKS Wall St flat as data offsets profit ...,2009-07-16
4,US STOCKS Wall St opens flat as data offsets e...,2009-07-17
...,...,...
5166,Snap Earnings Preview Stability In Revenue Is...,2020-02-05
5167,Potential Chinese Stimulus Helps Keep Wall Str...,2020-02-05
5168,Alphabet Slips As Revenue Growth Slowest In 4 ...,2020-02-05
5169,Forget Google s Advertising Business Cloud Is...,2020-02-05


In [119]:
# Instantiate the sentiment intensity analyzer
vader = SentimentIntensityAnalyzer()

In [120]:
# Iterate through the headlines and get the polarity scores using vader
scores = df['headline'].apply(vader.polarity_scores).tolist()
scores

[{'neg': 0.31, 'neu': 0.69, 'pos': 0.0, 'compound': -0.5574},
 {'neg': 0.0, 'neu': 0.522, 'pos': 0.478, 'compound': 0.6705},
 {'neg': 0.0, 'neu': 0.584, 'pos': 0.416, 'compound': 0.6908},
 {'neg': 0.0, 'neu': 0.556, 'pos': 0.444, 'compound': 0.7506},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.189, 'neu': 0.811, 'pos': 0.0, 'compound': -0.2732},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.23, 'neu': 0.556, 'pos': 0.214, 'compound': -0.0516},
 {'neg': 0.0, 'neu': 0.806, 'pos': 0.194, 'compound': 0.34},
 {'neg': 0.201, 'neu': 0.556, 'pos': 0.243, 'compound': 0.1531},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.196, 'neu': 0.652, 'pos': 0.152, 'compound': -0.1531},
 {'neg': 0.196, 'neu': 0.652, 'pos': 0.152, 'compound': -0.1531},
 {'neg': 0.0, 'neu': 1.0, 

In [121]:
# Convert the 'scores' list of dicts into a DataFrame
scores_df = pd.DataFrame(scores)
scores_df

,neg,neu,pos,compound
0,0.310,0.690,0.000,-0.5574
1,0.000,0.522,0.478,0.6705
2,0.000,0.584,0.416,0.6908
3,0.000,0.556,0.444,0.7506
4,0.000,1.000,0.000,0.0000
...,...,...,...,...
5166,0.000,1.000,0.000,0.0000
5167,0.000,0.776,0.224,0.3818
5168,0.000,0.729,0.271,0.3818
5169,0.192,0.808,0.000,-0.2263


In [122]:
# Join the DataFrames of the news and the list of dicts
scores_df = df.join(scores_df, rsuffix='_right')
scores_df

,headline,date,neg,neu,pos,compound
0,Get SMS ing U S tech execs urge Web starved ...,2009-04-22,0.310,0.690,0.000,-0.5574
1,CORRECTED Little known cloud startups attrac...,2009-05-06,0.000,0.522,0.478,0.6705
2,US STOCKS Wall St climbs on tech shares earni...,2009-07-16,0.000,0.584,0.416,0.6908
3,US STOCKS Wall St flat as data offsets profit ...,2009-07-16,0.000,0.556,0.444,0.7506
4,US STOCKS Wall St opens flat as data offsets e...,2009-07-17,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...
5166,Snap Earnings Preview Stability In Revenue Is...,2020-02-05,0.000,1.000,0.000,0.0000
5167,Potential Chinese Stimulus Helps Keep Wall Str...,2020-02-05,0.000,0.776,0.224,0.3818
5168,Alphabet Slips As Revenue Growth Slowest In 4 ...,2020-02-05,0.000,0.729,0.271,0.3818
5169,Forget Google s Advertising Business Cloud Is...,2020-02-05,0.192,0.808,0.000,-0.2263


In [123]:
scores_df[scores_df.isnull().any(axis=1)]

,headline,date,neg,neu,pos,compound


In [124]:
scores_df = scores_df.sort_values(by=['date'], ascending=True)
scores_df

,headline,date,neg,neu,pos,compound
0,Get SMS ing U S tech execs urge Web starved ...,2009-04-22,0.310,0.690,0.000,-0.5574
1,CORRECTED Little known cloud startups attrac...,2009-05-06,0.000,0.522,0.478,0.6705
2,US STOCKS Wall St climbs on tech shares earni...,2009-07-16,0.000,0.584,0.416,0.6908
3,US STOCKS Wall St flat as data offsets profit ...,2009-07-16,0.000,0.556,0.444,0.7506
4,US STOCKS Wall St opens flat as data offsets e...,2009-07-17,0.000,1.000,0.000,0.0000
...,...,...,...,...,...,...
5150,NVIDIA Officially Launches Google Stadia Compe...,2020-02-05,0.000,1.000,0.000,0.0000
5149,Intel bets on smart buildings in Israel to att...,2020-02-05,0.000,0.500,0.500,0.7906
5148,3 In Demand Software Stocks Worth Considering ...,2020-02-05,0.132,0.702,0.167,0.1027
5158,Alphabet Falls on Revenue Miss but Wall Stree...,2020-02-05,0.080,0.612,0.309,0.6956


In [125]:
mean_scores = scores_df.drop(columns = ['neg','neu','pos'])
mean_scores

,headline,date,compound
0,Get SMS ing U S tech execs urge Web starved ...,2009-04-22,-0.5574
1,CORRECTED Little known cloud startups attrac...,2009-05-06,0.6705
2,US STOCKS Wall St climbs on tech shares earni...,2009-07-16,0.6908
3,US STOCKS Wall St flat as data offsets profit ...,2009-07-16,0.7506
4,US STOCKS Wall St opens flat as data offsets e...,2009-07-17,0.0000
...,...,...,...
5150,NVIDIA Officially Launches Google Stadia Compe...,2020-02-05,0.0000
5149,Intel bets on smart buildings in Israel to att...,2020-02-05,0.7906
5148,3 In Demand Software Stocks Worth Considering ...,2020-02-05,0.1027
5158,Alphabet Falls on Revenue Miss but Wall Stree...,2020-02-05,0.6956


In [126]:
mean_scores = mean_scores.groupby(['date']).agg(['mean'])
mean_scores

,compound
,mean
date,
2009-04-22,-0.557400
2009-05-06,0.670500
2009-07-16,0.720700
2009-07-17,0.000000
2009-08-09,0.000000
...,...
2020-02-01,-0.318200
2020-02-02,0.018391


In [127]:
mean_scores['date'] = mean_scores.index
mean_scores = mean_scores.reset_index(drop=True)
mean_scores

,compound,date
,mean,
0,-0.557400,2009-04-22
1,0.670500,2009-05-06
2,0.720700,2009-07-16
3,0.000000,2009-07-17
4,0.000000,2009-08-09
...,...,...
1646,-0.318200,2020-02-01
1647,0.018391,2020-02-02
1648,0.118797,2020-02-03


In [128]:
mean_scores['date'] = pd.to_datetime(mean_scores['date'])
mean_scores = mean_scores.reset_index(drop=True)
mean_scores

,compound,date
,mean,
0,-0.557400,2009-04-22
1,0.670500,2009-05-06
2,0.720700,2009-07-16
3,0.000000,2009-07-17
4,0.000000,2009-08-09
...,...,...
1646,-0.318200,2020-02-01
1647,0.018391,2020-02-02
1648,0.118797,2020-02-03


In [129]:
mean_scores2 = pd.DataFrame(columns = {'date','mean_compound'})
mean_scores2['date'] = mean_scores['date']
mean_scores2['mean_compound'] = mean_scores['compound']['mean']
mean_scores2

,mean_compound,date
0,-0.557400,2009-04-22
1,0.670500,2009-05-06
2,0.720700,2009-07-16
3,0.000000,2009-07-17
4,0.000000,2009-08-09
...,...,...
1646,-0.318200,2020-02-01
1647,0.018391,2020-02-02
1648,0.118797,2020-02-03
1649,0.069568,2020-02-04


In [130]:
scores_name = "FinalData/vadar_scores_"+str(company)+".csv"
mean_scores_name = "FinalData/vadar_mean_compound_scores_"+str(company)+".csv"

In [131]:
scores_df.to_csv(scores_name)

In [132]:
mean_scores2.to_csv(mean_scores_name)